In [1]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import numpy as np

from PyPDF2 import utils
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser, PDFSyntaxError
from pdfminer.pdfdocument import PDFDocument
from requests.exceptions import MissingSchema

import datetime as dt
from time import mktime, strptime

from io import StringIO
import io

from typing import Tuple

In [2]:
class BankTrackDataProcessor:
    
    def __init__(self):
        
        # get the json
        self.bt_dict = dict(requests.get(BT_JSON_URL).json())

    def get_bank_links(self) -> pd.DataFrame:
        """"""
        # get bank links
        df_bank_links = pd.DataFrame.from_dict(self.bt_dict['csrdocs'])
        
        # get external link if available, otherwise internal
        df_bank_links['link'] = np.where(df_bank_links['link_external'] == '', 
                                         df_bank_links['link_internal'],
                                         df_bank_links['link_external'])
        
        df_bank_links = df_bank_links.explode('bp_ids')
        return df_bank_links

    def get_bank_id(self) -> int:
        """"""
        # get bank meta
        bank_meta = pd.DataFrame.from_dict(self.bt_dict['bps'], orient='index')
        bank_id = bank_meta.query("title == @BANK_NAME").id.squeeze()
        return bank_id


def get_match_score(string: str) -> int:
    """
    returns the number of policy keywords contained
    by the input string
    """

    match_score = 0
    string = string.lower()
    for kw in POLICY_KEYWORDS:
        match_score += kw in string
        
    return match_score

def convert_pdf_to_txt(pdf_link: str, pages=None) -> str:
    """Converts pdf to text"""
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    r = requests.get(pdf_link)
    f = io.BytesIO(r.content)

    for page in PDFPage.get_pages(f, pagenums):
        interpreter.process_page(page)
    converter.close()
    text = output.getvalue()
    output.close()
    return text


def get_pdf_meta(pdf_link: str) -> Tuple[str, str]:
    """Given a pdf link, returns the first 50 characters and
    the creation date, if available
    """
    try:
        txt = convert_pdf_to_txt(pdf_link, pages=[0])
    except (utils.PdfReadError, PDFSyntaxError) as e:
        print(f"can't read {pdf_link}")
        return np.nan, np.nan

    r = requests.get(pdf_link)
    f = io.BytesIO(r.content)

    parser = PDFParser(f)
    doc = PDFDocument(parser)
    try:
        creation_date = doc.info[0]["CreationDate"].decode("utf-8")

        creation_date = creation_date.rstrip("Z").split("+")[0].split("-")[
            0].rstrip("Z'")
        creation_date = str(dt.datetime.fromtimestamp(mktime(strptime(
            creation_date[2:], "%Y%m%d%H%M%S"))))
    except KeyError:
        print(f"Can't fetch date for PDF {pdf_link}")
        creation_date = 'NA'

    sample_txt = txt.replace("\n", '').strip(" ")[0:50]
    if sample_txt == '\x0c':
        print(f"Couldn't extract text from {pdf_link}. Likely the PDF is "
              f"protected against copy")

    return sample_txt, creation_date


def garnish_with_pdf_meta(link_df_in: pd.DataFrame, 
                          url_field: str) -> pd.DataFrame:
    """
    Adds pdf metadata to each pdf link in a dataframe.
    'sample_text' and 'created_at' fields are added
    """
    link_df = link_df_in.copy()
    
    # get pdf metadata
    meta_tpl = link_df[url_field].map(get_pdf_meta)
    
    # add metadata to the dataframe
    link_df['sample_text'] = [tpl[0] for tpl in meta_tpl]
    link_df['created_at'] = [tpl[1] for tpl in meta_tpl]

    na_mask = link_df['sample_text'].isna()
    if na_mask.any():
        print(f"{na_mask.sum()} missing values in sample text field. "
              f"Dropping these rows")
        link_df.dropna(subset=['sample_text'], inplace=True)
    
    return link_df

def process_scraped_links(dct_html_to_links) -> pd.DataFrame:

    # collect all the scraped htmls and pdfs in a dataframe
    df_scraped_links = pd.DataFrame.from_dict(dct_html_to_links, orient='index')

    # process the scraped links dataframe
    df_scraped_links['scraped_url'] = df_scraped_links['scraped_url'].map(list)

    df_scraped_links = (
        df_scraped_links.
        explode('scraped_url').
        pipe(lambda df: df.assign(
            filename = df['scraped_url'].str.split('/').str[-1]
        )
            ).
        reset_index().
        rename({'index': 'parent_html'}, axis=1)
    )

    return df_scraped_links

def collect_parent_htmls_per_file(df_scraped_links: pd.DataFrame) -> pd.DataFrame:
    
    # collect all parent htmls per file
    df_parent_htmls = (
        df_scraped_links.
        pipe(lambda df: df.assign(
            parent_html = df['parent_html'].str.replace('https:/', 'https://').str.replace('///', '//'),
            scraped_url = df['scraped_url'].str.replace('https:/', 'https://').str.replace('///', '//'))).
        groupby("filename").
        agg(parent_htmls=('parent_html', list)).
        reset_index()
    )

    # merge scraped links back to collected parent htmls using filename 
    df_scraped_links = (
        df_parent_htmls.
        merge(df_scraped_links.
              drop_duplicates('filename'), 
              on='filename', how='left').
        drop('parent_html', axis=1)
    )
    
    return df_scraped_links
    

## Notebook Description

This notebook provides prototype code for scraping through the bank's website and retrieving parent HTMLs of PDFs stored by BankTrack.

The script also retrieves any other HTMLs or PDFs that it thinks may be relevant based on a list of keywords.

The scraper is designed likes this:
1. Starts on some link (such as the ESG url linked by BankTrack). 
2. Collects all links from that page. 
3. If there are any PDFs that match keywords during step 2, then we save these and the parent HTML
4. The links are sorted, to put the ones that match `POLICY_KEYWORDS` at the front. 
5. Most relevant link is accessed

Steps 2-5 are repeated until all links were visited or we have visited a more than N links

### Setup

In [23]:
BT_JSON_URL = 'https://www.banktrack.org/service/sections/Document/csrdata'

POLICY_KEYWORDS = ('climate', 'esg', 'slave', 'palm', 'oil', 'coal', 
                   'forest', 'defence', 'environment', 'energy', 'bribery', 'corrupt', 
                   'human', 'rights', 'mine', 'mining', 'metal', 'power', 'waste','impact',
                   'mountain', 'sustain', 'fish', 'agricult', 'commodit', 'conduct')

BANK_NAME = 'Citi'
BASE_URL = "https://www.citigroup.com/" # must be the bare minimum needed to acceess the bank website
START_URLS = {'https://www.citigroup.com/citi/sustainability/policies.htm'} # should be the ESG link(s) linked by banktrack

bank_domain = BASE_URL.split('//')[-1]

### Get BankTrack stored links for the bank

In [24]:
# get a dataframe of banktrak links for the chosen bank
bt_data_processor = BankTrackDataProcessor()

bank_id = bt_data_processor.get_bank_id()

df_banktrack_links = bt_data_processor.get_bank_links()
df_banktrack_links = df_banktrack_links.query("bp_ids == @bank_id")
df_banktrack_links.head(3)

,id,tag,date,title,link_external,link_internal,file,pubpriv,bp_ids,link
109,10979,16af3ec,2014-01-01 00:00:00,Sector briefs - Thermal power,,https://www.banktrack.org/download/16af3ec,1160856_sector_brief_thermal_power.pdf,pub,50,https://www.banktrack.org/download/16af3ec
129,11279,forestry_pdf,2013-12-31 00:00:00,Sustainable Forestry Standard,,https://www.banktrack.org/download/forestry_pdf,forestry.pdf,pub,50,https://www.banktrack.org/download/forestry_pdf
156,11677,sustainable_progress_citi_s_five_year_sustaina...,2015-02-18 00:00:00,Sustainable Progress,,https://www.banktrack.org/download/sustainable...,Sustainable Progress Citi's five year sustaina...,pub,50,https://www.banktrack.org/download/sustainable...


In [25]:
print(f"Collecting PDF metadata for {len(df_banktrack_links)} PDFs")
df_banktrack_links = garnish_with_pdf_meta(df_banktrack_links, url_field='link')

can't read https://www.citigroup.com/citi/sustainability/policies.htm
1 missing values in sample text field. Dropping these rows


### Scrape HTMLs and PDFs from bank's website

In [26]:
# this dict will store the scraped pdf 
dct_html_to_links = defaultdict(lambda: defaultdict(lambda: set()))

# set start values
start_url = list(START_URLS)[0]
response = requests.get(start_url)
print(f"Starting with {start_url}")

unique_urls = START_URLS
visited_urls = set()
counter = 0

while len(unique_urls) > len(visited_urls):
        
    counter += 1
    if (counter % 100) == 0:
        print(f"Fetching {response.url} url")
        print(f"Visited {len(visited_urls)} out of "
              f"{len(unique_urls)} unique links")
        
        if len(unvisited_urls) <= 1:
            print("Visited all URLs. Stopping the scraper.")
            break
    
    # get soup from response
    try:
        soup = BeautifulSoup(response.text, "html.parser")
    except TypeError as e:
        pass
    
    # loop through all a tags, extracting links 
    for link in soup.find_all("a", href=True):
        try:
            url = link["href"]
        except:
            continue
        
        # pre-pend bank domain to a link if it's not on the banks 
        # website. This is a hacky way to only stay on the 
        # bank's website. Can be optimised further
        if bank_domain not in url:
            absolute_url = BASE_URL + url
        else:
            absolute_url = url
        
        # add collected link
        unique_urls.add(absolute_url)
        
        # if link is a pdf AND matches at least one keyword, we store it.
        # The link is stored under with the html page as the key
        if ('.pdf' in absolute_url) & (get_match_score(absolute_url) > 0):
            dct_html_to_links[response.url]['scraped_url'].add(absolute_url)
    
    # get the urls that we still didn't visit
    unvisited_urls = (unique_urls - visited_urls)
    
    # sort unvisited urls, such that we prioritise those that 
    # match most keywords
    unvisited_urls = sorted(list(unvisited_urls), 
                            key=get_match_score, reverse=False)
    
    # define the next url to visit
    unvisited_url = unvisited_urls.pop()
    
    for _ in range(len(unvisited_urls)):
        
        # while the next url to visit is a pdf, zip or xlsx, then we don't follow it
        # and choose another url.
        # it can slow things down a lot to scan through a large PDF.
        if ('.pdf' in unvisited_url) or unvisited_url.endswith('zip') or \
        unvisited_url.endswith('PDF') or unvisited_url.endswith('xlsx') or \
        unvisited_url.endswith('jpg'):
            
            visited_urls.add(unvisited_url)
            unvisited_url = unvisited_urls.pop()
            
        else:
            # if next url is an html, we fetch response
            # and add it to the set of visited urls
            try:
                response = requests.get(unvisited_url)
            except MissingSchema:
                visited_urls.add(unvisited_url)
                unvisited_url = 'https:' + unvisited_url
                response = requests.get(unvisited_url)
                unique_urls.add(unvisited_url)
                
            visited_urls.add(unvisited_url)
            break

    if len(visited_urls) > 2000:
        print("Visited over 3000 URLs. Stopping the scraper.")
        break

Starting with https://www.citigroup.com/citi/sustainability/policies.htm
Fetching https://www.citigroup.com/citi/news/2020/200520b.htm url
Visited 116 out of 207 unique links
Fetching https://www.citigroup.com/mailto:shareholder@computershare.com url
Visited 315 out of 428 unique links
Fetching https://www.citigroup.com/city_administration/cs_unitedstates.htm url
Visited 462 out of 547 unique links
Fetching https://www.citigroup.com/citi/#modal/citi-turning-conversations-into-creative-solutions url
Visited 562 out of 613 unique links
Fetching https://www.citigroup.com/citi/news/2020/200917a.htm url
Visited 624 out of 625 unique links
Visited all URLs. Stopping the scraper.


### Process scraped links

In [27]:
df_scraped_links = process_scraped_links(dct_html_to_links)
df_scraped_links = collect_parent_htmls_per_file(df_scraped_links)

print(f"Collecting PDF metadata for {len(df_scraped_links)} PDFs")
df_scraped_links = garnish_with_pdf_meta(df_scraped_links, url_field='scraped_url')

can't read https://www.citigroup.com/download/2016/2016-Brazil-Sustainability-Report.pdf
can't read https://www.citigroup.com/data/CitiGreenBondSPO-SustainalyticsOpinion.pdf
can't read http://www.citigroup.com/citi/environment/data/Corporate_Sustainability_Strategy.pdf
3 missing values in sample text field. Dropping these rows


In [28]:
# merge scraped links to banktrack links using pdf text
df_banktrack_links = df_banktrack_links.drop_duplicates('sample_text')
df_scraped_links = df_scraped_links.drop_duplicates('sample_text')

df_all_links = df_banktrack_links.merge(df_scraped_links, on='sample_text', 
                                        how='outer', indicator=True)

### Check matches and other potentially relevant links

In [29]:
n_banktrack_links = len(df_banktrack_links)

#### Check matches

In [30]:
df_matches = df_all_links.query("_merge == 'both'")
print(f"Found parent htmls for {len(df_matches)} out of {n_banktrack_links} BankTrack stored docs\n")
print(f"These are: {df_matches['scraped_url'].to_list()}\n")


match_parent_htmls = set(df_matches['parent_htmls'].explode())
print("Found the following parent HTMLs for these PDFs:")
for phtml in match_parent_htmls:
    print(phtml)

Found parent htmls for 3 out of 11 BankTrack stored docs

These are: ['https://www.citigroup.com//citi/investor/data/codeconduct_en.pdf', 'https://www.citigroup.com//citi/citizen/data/citi_statement_on_human_rights.pdf', 'https://www.citigroup.com//citi/sustainability/data/Environmental-and-Social-Policy-Framework.pdf']

Found the following parent HTMLs for these PDFs:
https://www.citigroup.com//citi/fixedincome/green_bonds.htm
https://www.citigroup.com//citi/sustainability/climaterisk.htm
https://www.citigroup.com/citi/sustainability/policies.htm
https://www.citigroup.com//citi/sustainability/lowcarbon.htm
https://www.citigroup.com//citi/sustainability/operations.htm
https://www.citigroup.com//citi/sustainability/


#### Check PDFs only found on Banktrack

In [31]:
df_bt_only = df_all_links.query("_merge == 'left_only'")
print(f"Did not find parent htmls for {len(df_bt_only)} "
      f"out of {len(df_bt_only) + len(df_matches)} BankTrack stored docs\n")

for pdf in df_bt_only['link']:
    print(pdf)

print("\nNOTE it might be that for some of these PDFs simply the newer version of the doc was scraped. "
      "Hence simply the outdated PDF versions stored by BankTrack are not stored by bank anymore")

Did not find parent htmls for 8 out of 11 BankTrack stored docs

https://www.banktrack.org/download/16af3ec
https://www.banktrack.org/download/forestry_pdf
https://www.banktrack.org/download/sustainable_progress_citi_s_five_year_sustainability_strategy_pdf
https://www.banktrack.org/download/citi_antibribery_program
https://www.banktrack.org/download/uk_modern_slavery_act_statement
https://www.banktrack.org/download/sector_briefs
https://www.banktrack.org/download/code_of_ethics_for_financial_professionals
https://www.banktrack.org/download/modern_slavery_statement_7

NOTE it might be that for some of these PDFs simply the newer version of the doc was scraped. Hence simply the outdated PDF versions stored by BankTrack are not stored by bank anymore


#### Check potentially interesting PDFs and HTMLs not found on BankTrack

In [32]:
df_scraped_only = df_all_links.query("_merge == 'right_only'")
scraped_pdfs = set(df_scraped_only['scraped_url'])

pdfs_relevancy_mask = np.array(list(map(get_match_score, scraped_pdfs))).nonzero()
scraped_pdfs = np.array(list(scraped_pdfs))[pdfs_relevancy_mask]

scraped_pdfs = sorted(scraped_pdfs, 
                      key=get_match_score, reverse=True)

print(f"The following scraped PDFs were not stored by BankTrack, but match "
      f"keywords of interest and may be good to track.\n"
      f"Sorted from most relevant to least relevant.\n")
for pdf in scraped_pdfs:
    print(pdf)

The following scraped PDFs were not stored by BankTrack, but match keywords of interest and may be good to track.
Sorted from most relevant to least relevant.

https://www.citigroup.com//citi/sustainability/data/Impact-Accounting-Methodology.pdf
https://www.citigroup.com//citi/sustainability/data/finance-for-a-climate-resilient-future.pdf
https://www.citigroup.com//citi/about/esg/download/2019/Executive-Summary-2019.pdf
https://www.citigroup.com//citi/investor/data/uk_modern_slavery_statement_2019.pdf
https://www.citigroup.com//citi/sustainability/data/Financial-Accounting-Methodology.pdf
https://www.citigroup.com//citi/foundation/data/2020-Pathways-Impact-Report.pdf
https://www.citigroup.com//citi/about/esg/download/2019/Global-ESG-Report-2019.pdf
https://www.citigroup.com/citi/fixedincome/data/opinion_by_sustainalytics.pdf


In [34]:
htmls_of_scraped_pdfs = \
    set(df_all_links[df_all_links['scraped_url'].isin(scraped_pdfs)]['parent_htmls'].explode())

scraped_htmls = set(df_scraped_only['parent_htmls'].explode())
scraped_htmls = scraped_htmls - match_parent_htmls

scraped_htmls = sorted(scraped_htmls, 
                       key=get_match_score, reverse=True)

scraped_htmls = htmls_of_scraped_pdfs.union(scraped_htmls)

htmls_relevancy_mask = np.array(list(map(get_match_score, scraped_htmls))).nonzero()
scraped_htmls = np.array(list(scraped_htmls))[htmls_relevancy_mask]

scraped_htmls = set([html for html in scraped_htmls if 'news' not in html])

print(f"The following scraped HTMLS were not found when checking parent pages for PDFs stored by BankTrack, but match keywords of "
      f"interest and may be good to track.\n"
      f"Sorted from most relevant to least relevant.\n")
for phtml in scraped_htmls - match_parent_htmls:
    print(phtml)

The following scraped HTMLS were not found when checking parent pages for PDFs stored by BankTrack, but match keywords of interest and may be good to track.
Sorted from most relevant to least relevant.

https://www.citigroup.com//citi/sustainability/evolution.htm#Equator
https://www.citigroup.com//citi/sustainability/evolution.htm#green
https://www.citigroup.com/citi/about/citizenship/citi-impact-fund.html


### Observations
1. By a large mergin, the most links on BankTrack are stored internally and don't have an external link right now
2. Some pdfs can't be read. One reason is because sometimes the href is relative. Currently the script just prepends the base url, but this doesn't always work. We need to extract the url of the current page and prepend this.
3. Some PDFs are locked for copy. The method that is used currently fails to convert it to text because of this. So we can't match these PDFs on text.
4. There are some relevant PDFs and HTMLs returned by the scraper, but not all are relevant. Can be stricter with keywords to filter these out. Especially true for banks like Barclays, that return a lot of relevant links. Links for parent HTMLs of PDFs that are stored on BankTrack, are definitely relevant. Then there are also HTMLs and PDFs that were not found on BankTrack, but may be relevant. Some of these PDFs are for docs that bank track does store, but the updated version, e.g. BankTrack might have Human-Rights-2020 but we scrape Human-Rights-2021 from bank's website
5. Sometimes the BankTrack stored PDFs seemingly cannot be found on the banks website. Some of these cases could be attributed that a new version is available, as stated in point 4. Perhaps some PDFs are only accessible through a google site search